In [1]:
import pandas as pd
tweets = pd.read_csv('tweets_premier_1819.csv')
# remove duplicated tweets
tweets = tweets.drop_duplicates(['tweet_id']).reset_index(drop=True)
tweets = tweets.drop(['tweet_text','tweet_id','tweet_username','tweet_date','tweet_mentions','tweet_hashtags','tweet_geo'], axis=1)
tweets.head()

,match_round,home_team,away_team,team,tweet_retweets,tweet_favorites
0,38,Brighton,Manchester City,Brighton,0,2
1,38,Brighton,Manchester City,Brighton,8,40
2,38,Brighton,Manchester City,Brighton,0,0
3,38,Brighton,Manchester City,Brighton,0,0
4,38,Brighton,Manchester City,Brighton,0,0


In [2]:
feature = tweets.groupby(['match_round','home_team','away_team','team']).sum().reset_index()
feature.head(10)

,match_round,home_team,away_team,team,tweet_retweets,tweet_favorites
0,1,Arsenal,Manchester City,Arsenal,2288,11238
1,1,Arsenal,Manchester City,Manchester City,4301,28705
2,1,Bournemouth,Cardiff,Bournemouth,819,4237
3,1,Bournemouth,Cardiff,Cardiff,3407,17304
4,1,Fulham,Crystal Palace,Crystal Palace,916,5774
5,1,Fulham,Crystal Palace,Fulham,1584,8355
6,1,Huddersfield,Chelsea,Chelsea,1419,7867
7,1,Huddersfield,Chelsea,Huddersfield,1479,7046
8,1,Liverpool,West Ham,Liverpool,1103,5738
9,1,Liverpool,West Ham,West Ham,570,2888


In [3]:
calendar = feature[['match_round', 'home_team', 'away_team']].drop_duplicates().reset_index(drop=True)
calendar.head()

,match_round,home_team,away_team
0,1,Arsenal,Manchester City
1,1,Bournemouth,Cardiff
2,1,Fulham,Crystal Palace
3,1,Huddersfield,Chelsea
4,1,Liverpool,West Ham


In [ ]:
final_df = pd.merge(calendar, feature.drop(['home_team','away_team'], axis=1), how='left', right_on = ['match_round', 'team'], left_on = ['match_round', 'home_team'])
final_df = pd.merge(final_df, feature.drop(['home_team','away_team'], axis=1), how='left', right_on = ['match_round', 'team'], left_on = ['match_round', 'away_team'])
final_df = final_df.drop(['team_x','team_y'], axis=1)
final_df = final_df.rename(columns={'tweet_retweets_x':'tweet_retweets_home', 
                            'tweet_favorites_x': 'tweet_favorites_home',
                            'tweet_retweets_y':'tweet_retweets_away', 
                            'tweet_favorites_y': 'tweet_favorites_away',
                           })

In [12]:
final_df

,match_round,home_team,away_team,tweet_retweets_home,tweet_favorites_home,tweet_retweets_away,tweet_favorites_away
0,1,Arsenal,Manchester City,2288,11238,4301,28705
1,1,Bournemouth,Cardiff,819,4237,3407,17304
2,1,Fulham,Crystal Palace,1584,8355,916,5774
3,1,Huddersfield,Chelsea,1479,7046,1419,7867
4,1,Liverpool,West Ham,1103,5738,570,2888
5,1,Manchester Utd,Leicester,1301,5564,981,4619
6,1,Newcastle,Tottenham,429,2296,919,4868
7,1,Southampton,Burnley,1222,11887,983,3431
8,1,Watford,Brighton,946,5332,1824,7809
9,1,Wolves,Everton,445,3360,815,5913


In [13]:
df_results = pd.read_csv('premier_lgb_ready.csv')
df_results = df_results[df_results['league_year']==2018].reset_index(drop=True)[['home_team','away_team','label']]

In [14]:
dataset = pd.merge(final_df, df_results)
dataset.head()

,match_round,home_team,away_team,tweet_retweets_home,tweet_favorites_home,tweet_retweets_away,tweet_favorites_away,label
0,1,Arsenal,Manchester City,2288,11238,4301,28705,2
1,1,Bournemouth,Cardiff,819,4237,3407,17304,1
2,1,Fulham,Crystal Palace,1584,8355,916,5774,2
3,1,Huddersfield,Chelsea,1479,7046,1419,7867,2
4,1,Liverpool,West Ham,1103,5738,570,2888,1


In [15]:
dataset.to_csv('twitter_retweets_favs_2018.csv', index=False)

In [16]:
# let's transform the teams in onehot encoding
onehot_home_names = pd.get_dummies(dataset.home_team, prefix = "Home_Team_")
new_ds = pd.concat([dataset, onehot_home_names], axis=1).drop(['home_team'], axis=1)
onehot_away_names = pd.get_dummies(dataset.away_team, prefix = "Away_Team_")
final_ds = pd.concat([new_ds, onehot_away_names], axis=1).drop(['away_team'], axis=1)

In [17]:
final_ds.shape

(380, 46)

In [18]:
# the test set is composed of the last 50 games of the EPL
X_train = final_ds.drop(['label'], axis=1)[:330]
X_test = final_ds.drop(['label'], axis=1)[-50:]
y_train = final_ds['label'][:330]
y_test = final_ds['label'][-50:]

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression(penalty='l2')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.50


C:\Users\BlueAce17\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\BlueAce17\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
